In [1]:
import os
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.autograd import Variable

import matplotlib as plt
from data import data_preprocess, data_trans
from modelbase import STA_LSTM as Net
# from modelbase import SA_LSTM as Net
# from modelbase import TA_LSTM as Net
# from modelbase import LSTM as Net
# from modelbase import FCN as Net
# from modelbase import SVM as Net

%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\yuman\anaconda3\envs\pytorch_v1\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
def train(verbose = False):

    net.train()
    loss_list = []
    alphas_list, betas_list = [],[]
    
    for i,data in enumerate(train_dataloader):
       
        inputs = data['inputs']
        groundtruths = data['groundtruths']     
        
        if USE_GPU:
            inputs = Variable(inputs).cuda()
            groundtruths = Variable(groundtruths).cuda()
            
        else:
            inputs = Variable(inputs)
            groundtruths = Variable(groundtruths)
        
        #将参数的grad值初始化为0
        optimizer.zero_grad()

        #获得网络输出结果
        out, alphas, betas = net(inputs)
        
        alphas_list.append(alphas)
        betas_list.append(betas)
        
        #根据真值计算损失函数的值
        loss = loss_criterion(out,groundtruths)

        #通过优化器优化网络
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    
    return loss_list, alphas_list, betas_list

def vali(verbose = False):

    net.eval()
    loss_list = []

    for i,data in enumerate(train_dataloader):
       
        inputs = data['inputs']
        groundtruths = data['groundtruths']     
        
        if USE_GPU:
            inputs = Variable(inputs).cuda()
            groundtruths = Variable(groundtruths).cuda()
            
        else:
            inputs = Variable(inputs)
            groundtruths = Variable(groundtruths)

        #获得网络输出结果
        out, _, _ = net(inputs)
        
        #根据真值计算损失函数的值
        loss = loss_criterion(out,groundtruths)
        loss_list.append(loss.item())
      
    return loss_list

def test():
    
    error = 0.0
    predictions = []
    test_groundtruths = []

    # 告诉网络进行测试，不再是训练模式
    net.eval() 

    for i,data in enumerate(test_dataloader):

        inputs = data['inputs']
        groundtruths = data['groundtruths']     
        
        if USE_GPU:

            inputs = Variable(inputs).cuda()
            groundtruths = Variable(groundtruths).cuda()
            
        else:
            
            inputs = Variable(inputs)
            groundtruths = Variable(groundtruths)

        out, _, _ = net(inputs)
        error += (error_criterion(out,groundtruths).item()*groundtruths.size(0))

        if USE_GPU:
            predictions.extend(out.cpu().data.numpy().tolist())
            test_groundtruths.extend(groundtruths.cpu().data.numpy().tolist())
            
        else:
            predictions.extend(out.data.numpy().tolist())
            test_groundtruths.extend(groundtruths.data.numpy().tolist())
      
    average_error = np.sqrt(error/len(test_data_trans))
    
    return np.array(predictions).reshape((len(predictions))),np.array(test_groundtruths).reshape((len(test_groundtruths))),average_error

Convert from each station's file to sample_aq_t+1.csv

In [3]:
SEQUENCE_LENGTH = 12   # 时间序列长度，即为回溯期
prediction_horizon = 2

'''****************************AQ data preparation*******************************'''
sensors = pd.read_csv('D:/Code/Code-AQ/airnow/processed_airnow/sensors_ca_preprocessed.csv', index_col=False)

fpath = "D:/Code/Code-Smoke-AQ/processed_data/airnow_smoke_fire/"
PM25_files = [fpath+f for f in os.listdir(fpath) if f.endswith('.csv')]

# larger la
PM25_files = [fpath+f for f in os.listdir(fpath) if f.startswith('06037') or f.startswith('06059') or f.startswith('06065') or f.startswith('06111')]
PM25_files

fname = PM25_files[0]
data = pd.read_csv(fname)
data = data[26305:52585] # 2017-2019
data = data.drop(columns=['datetime','u10','v10'])

cols = list(data.columns[:])
cols_PM25 = [col for col in cols if 'PM2.5' in col]
data[cols_PM25] = data[cols_PM25].fillna(method='ffill').fillna(method='bfill')

for col in data.columns:
    data[col] = data[col].fillna(method='ffill').fillna(method='bfill')

############################# include nearby sensors into data
sensor = fname.split('/')[-1].split('.')[0]
sensor_lon = sensors.loc[sensors['AQSID'] == int(sensor),'longitude'].values[0]
sensor_lat = sensors.loc[sensors['AQSID'] == int(sensor),'latitude'].values[0]
# print(sensor, sensor_lon, sensor_lat)

distances = {}
for file in PM25_files:
    if file != fname:
        # print(file, fname)
        sensor_near = file.split('/')[-1].split('.')[0]
        sensor_lon_near = sensors.loc[sensors['AQSID'] == int(sensor_near),'longitude'].values[0]
        sensor_lat_near = sensors.loc[sensors['AQSID'] == int(sensor_near),'latitude'].values[0]
        # print(sensor_near, sensor_lon_near, sensor_lat_near)
        dist = sqrt( (sensor_lon_near - sensor_lon)**2 + (sensor_lat_near - sensor_lat)**2 )
        if dist <= 0.4:
            distances[file] = dist

# print(distances)

for file in distances.keys():
    temp = pd.read_csv(file)
    temp = temp[26305:52585] # 2017-2019
    temp = temp[[[col for col in temp.columns if 'PM2.5' in col][0]]]
    temp.columns = [file.split('/')[-1].split('.')[0]]
    temp = temp.fillna(method='ffill').fillna(method='bfill')

    data[file.split('/')[-1].split('.')[0]] = temp

cols = list(data.columns[:])
cols_PM25 = [col for col in cols if 'PM2.5' in col]

############################# end include nearby sensors into data
data.head()

,blh,d2m,sp,t2m,smoke,fire,060370016_PM2.5,wind_dir,wind_spd,day_of_year,hour,day_of_week,quarter,holiday,060371103,060590007,060658005
26305,258.807146,279.257257,90259.986231,280.289850,0.0,0.0,1.4,219.634078,2.367715,1,1,6,1,False,4.0,3.3,3.8
26306,291.375577,278.787604,90275.489187,279.702898,0.0,0.0,3.8,240.017901,2.599394,1,2,6,1,False,3.0,4.5,5.0
26307,318.933480,278.112687,90290.992142,278.476318,0.0,0.0,19.9,262.584690,2.010053,1,3,6,1,False,2.0,6.9,8.2
26308,152.044361,277.936427,90304.356759,278.337105,0.0,0.0,22.5,241.357016,0.934233,1,4,6,1,False,3.0,10.3,8.2
26309,95.097786,277.556020,90282.438788,277.920720,0.0,0.0,35.3,175.670195,0.204549,1,5,6,1,False,8.0,24.0,5.8


In [4]:
# t0_v0, t0_v1, ..., t1_v0, t1_v1, .... , th_v0, th_v1, ...., t+h+1_target 
data_ = np.zeros((len(data), len(cols)*SEQUENCE_LENGTH + 1)) 

for i in range(SEQUENCE_LENGTH):
    data_[:, len(cols)*i:len(cols)*(i+1)] = data.shift(-i-1).fillna(method='bfill')
    
data_[:,-1] = data[cols_PM25].shift(-SEQUENCE_LENGTH - prediction_horizon).fillna(method='bfill').values.reshape(1,-1)

data_ = data_[:len(data)-SEQUENCE_LENGTH-prediction_horizon]

# Add all time points in the columns
column_names = []
for i in range(SEQUENCE_LENGTH):
    for var in cols:
        column_names.append(var+'_t'+str(i))
column_names.append('t+'+str(prediction_horizon))

df = pd.DataFrame(data_, columns=column_names)

savefp = './data/dataset/'+cols_PM25[0]+'_t'+str(SEQUENCE_LENGTH)+'_'+str(prediction_horizon)+'.csv'
df.to_csv(savefp,index=False)

In [5]:
'''****************************initialization*******************************''' 
IN_DIM =  len(cols)*SEQUENCE_LENGTH    # 因变量 number*SEQUENCE_LENGTH

LSTM_IN_DIM = len(cols)     # LSTM的input大小,等于总的变量长度/时间序列长度
LSTM_HIDDEN_DIM = 300  # LSTM隐状态的大小

OUT_DIM = 1            # 输出大小

LEARNING_RATE = 0.05 # learning rate
WEIGHT_DECAY = 1e-6    # L2惩罚项

BATCH_SIZE = 200        # batch size

EPOCHES = 180    # epoch大小

TRAIN_PER = 0.80 # 训练集占比
VALI_PER = 0.0 # 验证集占比

# 判断是否采用GPU加速
USE_GPU = torch.cuda.is_available()
# USE_GPU = False

In [6]:
'''****************************data prepration*******************************''' 
# 准备好训练和测试数据
dp = data_preprocess(file_path = savefp, train_per = TRAIN_PER, vali_per = VALI_PER, in_dim = IN_DIM)

raw_data = dp.load_data()
# print('数据导入完成')

(train_data,train_groundtruth),(vali_data,vali_groundtruth),(test_data,test_groundtruth) = dp.split_data(raw_data = raw_data, _type = 'linear')
# print('数据分割完成')

# 设置对数据进行的转换方式，transform.compose的作用是将多个transform组合到一起进行使用
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize(mean=(0,0,0),std=(1,1,1))])

# print('数据转换为tensor')

# data_trans返回的值是一个字典，内部包含数据和真值{'inputs':inputs,'groundtruth':groundtruths}

# 准备训练集
train_data_trans = data_trans(train_data,train_groundtruth,transform)

train_dataloader = torch.utils.data.DataLoader(train_data_trans,
                                           batch_size =BATCH_SIZE,
                                           shuffle = True,
                                           num_workers = 4)
# print('训练集准备完毕')

# 准备val集
vali_data_trans = data_trans(vali_data, vali_groundtruth, transform)

vali_dataloader = torch.utils.data.DataLoader(vali_data_trans,
                                           batch_size = BATCH_SIZE,
                                           shuffle = False,
                                           num_workers = 4)


# 准备测试集
test_data_trans = data_trans(test_data, test_groundtruth,transform)

test_dataloader = torch.utils.data.DataLoader(test_data_trans,
                                           batch_size = BATCH_SIZE,
                                           shuffle = False,
                                           num_workers = 4)
# print('测试集准备完毕')

In [7]:
raw_data.shape

(26266, 205)

In [8]:
raw_data

array([[ 2.91375577e+02,  2.78787604e+02,  9.02754892e+04, ...,
         9.30000000e+00,  3.15000000e+01, -1.60000000e+00],
       [ 3.18933480e+02,  2.78112687e+02,  9.02909921e+04, ...,
         1.24000000e+01,  3.55000000e+01, -1.00000000e-01],
       [ 1.52044361e+02,  2.77936427e+02,  9.03043568e+04, ...,
         1.76000000e+01,  3.17000000e+01,  7.90000000e+00],
       ...,
       [ 1.08375126e+02,  2.70912788e+02,  9.08184005e+04, ...,
         8.00000000e-01,  2.10000000e+00,  7.30000000e+00],
       [ 2.70219929e+02,  2.70924112e+02,  9.08005738e+04, ...,
         1.00000000e+00,  1.10000000e+00,  4.80000000e+00],
       [ 6.62851362e+01,  2.69857382e+02,  9.08283624e+04, ...,
         1.50000000e+00,  8.00000000e-01,  5.60000000e+00]])

In [9]:
train_data.shape

(21012, 204)

In [10]:
train_groundtruth.shape

(21012, 1)

In [11]:
test_groundtruth.shape

(5254, 1)

In [12]:
train_data_trans.__getitem__(0)['inputs']

tensor([2.9138e+02, 2.7879e+02, 9.0275e+04, 2.7970e+02, 0.0000e+00, 0.0000e+00,
        3.8000e+00, 2.4002e+02, 2.5994e+00, 1.0000e+00, 2.0000e+00, 6.0000e+00,
        1.0000e+00, 0.0000e+00, 3.0000e+00, 4.5000e+00, 5.0000e+00, 3.1893e+02,
        2.7811e+02, 9.0291e+04, 2.7848e+02, 0.0000e+00, 0.0000e+00, 1.9900e+01,
        2.6258e+02, 2.0101e+00, 1.0000e+00, 3.0000e+00, 6.0000e+00, 1.0000e+00,
        0.0000e+00, 2.0000e+00, 6.9000e+00, 8.2000e+00, 1.5204e+02, 2.7794e+02,
        9.0304e+04, 2.7834e+02, 0.0000e+00, 0.0000e+00, 2.2500e+01, 2.4136e+02,
        9.3423e-01, 1.0000e+00, 4.0000e+00, 6.0000e+00, 1.0000e+00, 0.0000e+00,
        3.0000e+00, 1.0300e+01, 8.2000e+00, 9.5098e+01, 2.7756e+02, 9.0282e+04,
        2.7792e+02, 0.0000e+00, 0.0000e+00, 3.5300e+01, 1.7567e+02, 2.0455e-01,
        1.0000e+00, 5.0000e+00, 6.0000e+00, 1.0000e+00, 0.0000e+00, 8.0000e+00,
        2.4000e+01, 5.8000e+00, 5.7808e+01, 2.7716e+02, 9.0303e+04, 2.7760e+02,
        0.0000e+00, 0.0000e+00, 1.4700e+

In [13]:
'''****************************model prepration*******************************''' 
# 将网络参数导入网络
net = Net(IN_DIM,SEQUENCE_LENGTH,LSTM_IN_DIM,LSTM_HIDDEN_DIM,OUT_DIM,USE_GPU)
# print('网络模型准备完毕')

# 判断GPU是否可用，如果可用则将net变成可用GPU加速的net
if USE_GPU:
    net = net.cuda()
    # print('本次实验使用GPU加速')
else:
    pass
    # print('本次实验不使用GPU加速')

# 使用SGD（随机梯度下降）优化，学习率为0.001，动量为0.9
# optimizer = optim.SGD(net.parameters(), lr= LEARNING_RATE, momentum=0.9) 
# 根据梯度调整参数数值，Adam算法
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# 学习率根据训练的次数进行调整
adjust_lr = optim.lr_scheduler.MultiStepLR(optimizer,
                                     milestones=[i*10 for i in range(EPOCHES//10)],
                                     gamma=0.5)

# 定义训练损失函数&测试误差函数
# loss_criterion = nn.SmoothL1Loss()
loss_criterion = nn.MSELoss()
error_criterion = nn.MSELoss()

In [14]:
#记录程序开始的时间
train_start = time.time()
loss_recorder = []

min_val_loss = 9999
patience = 10
counter = 0

print('starting training... ')

for epoch in range(EPOCHES):

    # adjust learning rate
    adjust_lr.step()

    train_loss_list, alphas_list, betas_list = train(verbose= True)
    train_loss = np.mean(train_loss_list)
    
    loss_recorder.append(train_loss)
       
    vali_loss = np.mean(vali())

    print('epoch = %d, train loss = %.5f, vali loss = %.5f'%(epoch+1,train_loss,vali_loss))
    
    if min_val_loss > vali_loss**0.5:
        min_val_loss = vali_loss**0.5
        print("Saving...")
        torch.save(net.state_dict(), './models/sta_lstm_'+cols_PM25[0]+'_t'+str(SEQUENCE_LENGTH)+'_'+str(prediction_horizon)+'.pt')
        
        # save the mean alphas and betas to csv if saving the model
        alphas = np.mean(np.array(alphas_list), axis=0)
        betas = np.mean(np.array(betas_list), axis=0)
        np.savetxt('./data/output/A_'+cols_PM25[0]+'_t'+str(SEQUENCE_LENGTH)+'_'+str(prediction_horizon)+'.csv', alphas, delimiter=',')
        np.savetxt('./data/output/B_'+cols_PM25[0]+'_t'+str(SEQUENCE_LENGTH)+'_'+str(prediction_horizon)+'.csv', betas, delimiter=',')
        
        counter = 0
    else: 
        counter += 1

    if counter == patience:
        break
    
print ('training time = {}s'.format(int((time.time() - train_start))))

net.load_state_dict(torch.load('./models/sta_lstm_'+cols_PM25[0]+'_t'+str(SEQUENCE_LENGTH)+'_'+str(prediction_horizon)+'.pt'))

# 记录测试开始的时间
test_start = time.time()
predictions, test_groundtruth, average_error = test()

print(predictions.shape)
print(test_groundtruth.shape)

print('test time = {}s'.format(int((time.time() - test_start)+1.0)))
print('average error = ',  average_error)

result = pd.DataFrame(data = {'Q(t+1)':predictions,'Q(t+1)truth':test_groundtruth})
result.to_csv('./data/output/out_'+cols_PM25[0]+'_t'+str(SEQUENCE_LENGTH)+'_'+str(prediction_horizon)+'.csv')

starting training... 


C:\Users\yuman\anaconda3\envs\pytorch_v1\lib\site-packages\torch\optim\lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


OUT!: torch.Size([200, 204])


D:\Code\Code-SpatioTemporalAttention-LSTM-main\modelbase.py:249: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  alpha_t = self.softmax(alpha_t)
D:\Code\Code-SpatioTemporalAttention-LSTM-main\modelbase.py:272: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  beta_t = self.softmax(beta_t)


OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Size([200, 204])
OUT!: torch.Si

KeyboardInterrupt: 

In [ ]:
def RMSE(v, v_):
    '''
    Mean squared error.
    :param v: np.ndarray or int, ground truth.
    :param v_: np.ndarray or int, prediction.
    :return: int, RMSE averages on all elements of input.
    '''
    return np.sqrt(np.mean((v_ - v) ** 2))

def MAE(v, v_):
    '''
    Mean absolute error.
    :param v: np.ndarray or int, ground truth.
    :param v_: np.ndarray or int, prediction.
    :return: int, MAE averages on all elements of input.
    '''
    return np.mean(np.abs(v_ - v))

print(cols_PM25[0]+'_t'+str(SEQUENCE_LENGTH)+'_'+str(prediction_horizon),', Test RMSE,', round(RMSE(test_groundtruth, predictions),4), ', Test MAE,', round(MAE(test_groundtruth, predictions),4))

In [ ]:
alphas = np.loadtxt('./models/A_'+cols_PM25[0]+'_t'+str(SEQUENCE_LENGTH)+'_'+str(prediction_horizon)+'.csv', delimiter=',')
alphas = np.transpose(alphas)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), dpi=300)
im = ax.imshow(alphas)
ax.set_xticks(np.arange(SEQUENCE_LENGTH))
ax.set_yticks(np.arange(len(cols)))
ax.set_xticklabels(["t-"+str(i) for i in np.arange(SEQUENCE_LENGTH, 0, -1)])
ax.set_yticklabels(cols)
# for i in range(len(cols)):
#     for j in range(SEQUENCE_LENGTH):
#         text = ax.text(j, i, round(alphas[i, j], 3),
#                        ha="center", va="center", color="w")
ax.set_title("Importance of features and timesteps")

# Calculate (height_of_image / width_of_image)
im_ratio = alphas.shape[0]/alphas.shape[1]
plt.colorbar(im, fraction=0.047*im_ratio, pad=0.01)
plt.savefig('./models/A_'+cols_PM25[0].split('_')[0]+'_t'+str(SEQUENCE_LENGTH)+'_'+str(prediction_horizon)+'.png')
plt.show()

Experiment result:

- 060370016_PM2.5_t72_1, Test RMSE, 3.9513 , Test MAE, 2.7456
- 060370016_PM2.5_t48_1 Test RMSE, 4.4159 , Test MAE, 2.6177
- 060370016_PM2.5_t36_1 , Test RMSE, 4.1832 , Test MAE, 2.6065
- 060370016_PM2.5_t12_1 , Test RMSE, 3.9024 , Test MAE, 2.5755